In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

data = pd.read_json('train.json')

In [3]:
data['num_ingredients']=data.ingredients.apply(lambda x:len(x))
data.head()

,id,cuisine,ingredients,num_ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...",9
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",11
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",12
3,22213,indian,"[water, vegetable oil, wheat, salt]",4
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...",20


In [4]:
data.groupby('cuisine').num_ingredients.mean().sort_values()

cuisine
irish            9.299850
brazilian        9.520343
southern_us      9.634954
british          9.708955
japanese         9.735067
french           9.817838
italian          9.909033
filipino        10.000000
greek           10.182128
russian         10.224949
spanish         10.423660
mexican         10.877446
korean          11.284337
chinese         11.982791
jamaican        12.214829
thai            12.545809
cajun_creole    12.617076
vietnamese      12.675152
indian          12.705961
moroccan        12.909866
Name: num_ingredients, dtype: float64

In [5]:
data.shape

(39774, 4)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import regexp_tokenize
from nltk.stem.snowball import SnowballStemmer
pd.set_option('display.width', 1000)
data['recipe']=data.ingredients.apply(lambda x:', '.join([(i.lower()) for i in x]))

In [7]:
data['recipe']=data.ingredients.apply(lambda x:', '.join([SnowballStemmer(language='english').stem(i.lower()) for i in x]))

In [8]:
#Given a set of ingredients find probability it is a particular cuisine
def find_probability_of_cuisine(lst,df):
    df['num_count']=df.ingredients.apply(lambda x:len(set(x).intersection(set(lst))))
    cross_t=pd.crosstab(df.cuisine,df.num_count).apply(lambda r: 100*r/r.sum(), axis=0)
#So maybe having multiple combinations 1s 2s and 3s of ingredients identify it best cartersian product of all the combinatinos
    reqd_df=pd.DataFrame(cross_t[len(lst)-1])
    reqd_df.columns=['perc']
    return reqd_df.perc.idxmax()

find_probability_of_cuisine(['garlic','onion','ginger','masala'],data)

'indian'

In [9]:
find_probability_of_cuisine(['garlic','onion','basil'],data)

'italian'

In [10]:
cv=CountVectorizer()
vectors=cv.fit_transform(data.recipe)
vectors

<39774x2974 sparse matrix of type '<class 'numpy.int64'>'
	with 760394 stored elements in Compressed Sparse Row format>

In [11]:
df=pd.DataFrame(data=vectors.todense(), columns=sorted(cv.vocabulary_))
df.head()

00  10  100  14  15  25  33  40  43  95  96  abalon  abbamel  absinth  abura  acai  accent  accompani  achiot  achiote  acid  acini  acke  acorn  acting  activ  active  added  adobo  adzuki  ag  agar  agave  aged  ahi  aioli  ajinomoto  ajwain  aka  al  alaskan  albacor  albacore  alcohol  ale  aleppo  alexia  alfalfa  alfredo  all  allspic  allspice  almond  almondmilk  aloe  alphabet  alum  amaranth  amarena  amaretti  amaretto  amba  amber  amberjack  amchur  america  american  amino  ammonium  amontillado  ampalaya  an  anaheim  anasazi  ancho  anchovi  anchovy  and  andouill  andouille  anejo  angel  anglais  angled  angostura  angus  ani  anis  anise  anisett  anjou  annatto  aonori  appl  apple  applesauc  applewood  apricot  aquavit  arak  aram  arbol  arborio  arctic  arepa  argo  arhar  armagnac  arrabbiata  arrow  arrowroot  artichok  artichoke  artificial  artisan  arugula  asada  asadero  asafetida  asafoetida  asakusa  ascorbic  asiago  asian  asparagus  aspic  \
0   0   0    0   0   0   0   0   0   0   0   0       0        0        0      0     0       0          0       0        0     0      0     0      0       0      0       0      0      0       0   0     0      0     0    0      0          0       0    0   0        0        0         0        0    0       0       0        0        0    0        0         0       0           0     0         0     0         0        0         0         0     0      0          0       0        0         0      0         0            0         0   0        0        0      0        0        0    0         0          0      0      0        0       0          0      0    0     0      0        0      0        0       0     0      0          0          0        0        0     0     0      0        0       0      0     0      0         0           0      0          0         0          0           0        0        0      0        0          0           0        0         0       0      0          0      0   
1   0   0    0   0   0   0   0   0   0   0   0       0        0        0      0     0       0          0       0        0     0      0     0      0       0      0       0      0      0       0   0     0      0     0    0      0          0       0    0   0        0        0         0        0    0       0       0        0        0    0        0         0       0           0     0         0     0         0        0         0         0     0      0          0       0        0         0      0         0            0         0   0        0        0      0        0        0    0         0          0      0      0        0       0          0      0    0     0      0        0      0        0       0     0      0          0          0        0        0     0     0      0        0       0      0     0      0         0           0      0          0         0          0           0        0        0      0        0          0           0        0         0       0      0          0      0   
2   0   0    0   0   0   0   0   0   0   0   0       0        0        0      0     0       0          0       0        0     0      0     0      0       0      0       0      0      0       0   0     0      0     0    0      0          0       0    0   0        0        0         0        0    0       0       0        0        0    0        0         0       0           0     0         0     0         0        0         0         0     0      0          0       0        0         0      0         0            0         0   0        0        0      0        0        0    0         0          0      0      0        0       0          0      0    0     0      0        0      0        0       0     0      0          0          0        0        0     0     0      0        0       0      0     0      0         0           0      0          0         0          0           0        0        0      0        0          0           0        0         0       0      0          0      0   
3   0   0    0   0   0 

In [12]:
from itertools import product,combinations
data['2_ingredient']=data.ingredients.apply(lambda x:list(combinations(x,2)))
data['3_ingredient']=data.ingredients.apply(lambda x:list(combinations(x,3)))
data.head()

,id,cuisine,ingredients,num_ingredients,recipe,num_count,2_ingredient,3_ingredient
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...",9,"romaine lettuc, black ol, grape tomato, garlic...",1,"[(romaine lettuce, black olives), (romaine let...","[(romaine lettuce, black olives, grape tomatoe..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",11,"plain flour, ground pepp, salt, tomato, ground...",0,"[(plain flour, ground pepper), (plain flour, s...","[(plain flour, ground pepper, salt), (plain fl..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",12,"egg, pepper, salt, mayonais, cooking oil, gree...",0,"[(eggs, pepper), (eggs, salt), (eggs, mayonais...","[(eggs, pepper, salt), (eggs, pepper, mayonais..."
3,22213,indian,"[water, vegetable oil, wheat, salt]",4,"water, vegetable oil, wheat, salt",0,"[(water, vegetable oil), (water, wheat), (wate...","[(water, vegetable oil, wheat), (water, vegeta..."
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...",20,"black pepp, shallot, cornflour, cayenne pepp, ...",0,"[(black pepper, shallots), (black pepper, corn...","[(black pepper, shallots, cornflour), (black p..."


In [13]:
class StemmerTokenizer(object):
    """
    Joins all ingredients together into a single string and provides
    a list of stems of all words longer than 2 letters.

   
    """

    def __init__(self):
        self.pattern = r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b'
        self.stemmer = SnowballStemmer('english')

    def mapper(self, word):
        return self.stemmer.stem(word)

    def tokenizer(self, doc):
        return [self.mapper(t) for t in regexp_tokenize(doc, pattern=self.pattern)]

    def preprocessor(self, line):
        return ' '.join(line).lower()
    
tok = StemmerTokenizer()
tok.tokenizer(
            tok.preprocessor([
                'romaine lettuce', 'black olives', 'grape tomatoes',
                'garlic', 'pepper', 'purple onion', 'seasoning',
                'garbanzo beans', 'feta cheese crumbles'
            ])
        )


class DupleTokenizer1(StemmerTokenizer):
    def tokenizer(self,doc):
        words = np.array([self.mapper(t) for t in regexp_tokenize(doc, pattern=self.pattern)])

        arr_2=list(combinations(words,2))
        
        words = sorted(set(words))
        all_w=[]
        for i in arr_2:
            (j,k)=i    
            wrd=(j+' '+k)
            all_w.append(wrd)
            
       
        return np.hstack([words,np.array(all_w)])
        

    
tok = DupleTokenizer1()
tok.tokenizer(
            tok.preprocessor([
               'black', 'pepper', 'salt', 'sugar'
            ])
        )


array(['black', 'pepper', 'salt', 'sugar', 'black pepper', 'black salt',
       'black sugar', 'pepper salt', 'pepper sugar', 'salt sugar'],
      dtype='<U12')

In [14]:
class StemmerTokenizer(object):
    """
    Joins all ingredients together into a single string and provides
    a list of stems of all words longer than 2 letters.

    Example:
    >>> tok = StemmerTokenizer()
    >>> tok.tokenizer(
            tok.preprocessor([
                'romaine lettuce', 'black olives', 'grape tomatoes',
                'garlic', 'pepper', 'purple onion', 'seasoning',
                'garbanzo beans', 'feta cheese crumbles'
            ])
        )
    ['romain', 'lettuc', 'black', 'oliv', 'grape', 'tomato',
     'garlic', 'pepper', 'purpl', 'onion', 'season',
     'garbanzo', 'bean', 'feta', 'chees', 'crumbl']
    """

    def __init__(self):
        self.pattern = r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b'
        self.stemmer = SnowballStemmer('english')

    def mapper(self, word):
        return self.stemmer.stem(word)

    def tokenizer(self, doc):
        return [self.mapper(t) for t in regexp_tokenize(doc, pattern=self.pattern)]

    def preprocessor(self, line):
        return ' '.join(line).lower()
class DupleTokenizer(StemmerTokenizer):
    """
    Builds upon the StemmerTokenizer: after all words in a recipe are stemmed
    they are grouped in all possible combinations of two words and added
    to the words' list.
    
    Example (simplified):
    >>> tok = DupleTokenizer()
    >>> tok.tokenizer(
            tok.preprocessor([
                'sugar', 'salt', 'black pepper'
            ])
        )
    ['black', 'pepper', 'salt', 'sugar',
     'black pepper', 'black salt', 'black sugar', 'pepper salt', 'pepper sugar', 'salt sugar']
    """
    def tokenizer(self, doc):
        
        def duple(words):
            duples = []
            i = 0
            while i < len(words)-1:
                j = i + 1
                while j < len(words):
                    duples.append('%s %s' % (words[i], words[j]))
                    j += 1
                i += 1
            return np.array(duples)
        
        words = np.array([self.mapper(t) for t in regexp_tokenize(doc, pattern=self.pattern)])
        words = sorted(set(words))
        words = np.hstack([words, duple(words)])
        return words

In [15]:
def features_count(tokenizer, df):
    vect = CountVectorizer(
        preprocessor=tokenizer().preprocessor,
        tokenizer=tokenizer().tokenizer)
    return len(vect.fit(df.ingredients).vocabulary_)

# It takes around 40s on my machine
data = pd.read_json('./train.json')
print ('StemmerTokenizer (single words):         ', features_count(StemmerTokenizer, data))
print ('DupleTokenizer (single words and pairs): ', features_count(DupleTokenizer1, data))

C:\Users\priya\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


StemmerTokenizer (single words):          2598
DupleTokenizer (single words and pairs):  439909


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

tokenizerClass = DupleTokenizer1
# Parameters were previously found based on a 3-fold cross validation
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(
        preprocessor=tokenizerClass().preprocessor,
        tokenizer=tokenizerClass().tokenizer,
        stop_words='english',
        max_df=1.0,
        min_df=1,
        binary=True,
    )),
    ('transformer', TfidfTransformer()),
    ('classifier', LinearSVC(
        C=0.78, penalty='l2', loss='squared_hinge', dual=True, max_iter=1000, random_state=0))
])

In [19]:
# Takes around 50s
train = pd.read_json('./train.json')
X_train = train['ingredients']
y_train = train['cuisine']

test = pd.read_json('./test.json')
X_test = test['ingredients']

pipeline.fit(X_train, y_train)
prediction = pipeline.predict(X_test)

In [20]:
submission = test.copy()
submission['cuisine'] = prediction
test1=submission
submission.to_csv(
    'submission.csv', index=False, quoting=3,
    columns=['id', 'cuisine'])

In [83]:
test1.to_csv("abc.csv",index=False)